In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate

# Data

## Load

In [ ]:
import pandas as pd
df = pd.read_json("/content/HS-Brexit_train.json")

In [ ]:
df = df.transpose()
df = df.rename(columns={"hard_label": "label"})

In [ ]:
df

,text,annotation task,number of annotations,annotations,annotators,lang,label,soft_label,split,other_info
1,<user> <user> I'm so glad about #Brexit.. My a...,hate speech detection,6,"0,0,0,0,0,0","Ann1,Ann2,Ann3,Ann4,Ann5,Ann6",en,0,"{'0': 1.0, '1': 0.0}",train,{'other annotations': {'aggressive language de...
2,RT <user>: There was more to #Brexit than immi...,hate speech detection,6,"0,0,0,0,0,0","Ann1,Ann2,Ann3,Ann4,Ann5,Ann6",en,0,"{'0': 1.0, '1': 0.0}",train,{'other annotations': {'aggressive language de...
3,"At the end of the day, the leave campaign won ...",hate speech detection,6,"0,0,0,0,0,0","Ann1,Ann2,Ann3,Ann4,Ann5,Ann6",en,0,"{'0': 1.0, '1': 0.0}",train,{'other annotations': {'aggressive language de...
4,So the reducing migration thing wasn't quite w...,hate speech detection,6,"0,0,0,0,0,0","Ann1,Ann2,Ann3,Ann4,Ann5,Ann6",en,0,"{'0': 1.0, '1': 0.0}",train,{'other annotations': {'aggressive language de...
5,A Brit Immigrant Asks Britain to Become India’...,hate speech detection,6,"0,0,0,0,0,0","Ann1,Ann2,Ann3,Ann4,Ann5,Ann6",en,0,"{'0': 1.0, '1': 0.0}",train,{'other annotations': {'aggressive language de...
...,...,...,...,...,...,...,...,...,...,...
780,#Brexit has to happen there is no way you can ...,hate speech detection,6,"0,0,0,0,0,0","Ann1,Ann2,Ann3,Ann4,Ann5,Ann6",en,0,"{'0': 1.0, '1': 0.0}",train,{'other annotations': {'aggressive language de...
781,A foreigner we've neither heard of nor voted f...,hate speech detection,6,"0,0,0,0,0,0","Ann1,Ann2,Ann3,Ann4,Ann5,Ann6",en,0,"{'0': 1.0, '1': 0.0}",train,{'other annotations': {'aggressive language de...
782,The irony is we will probably now look to appo...,hate speech detection,6,"0,0,0,0,0,0","Ann1,Ann2,Ann3,Ann4,Ann5,Ann6",en,0,"{'0': 1.0, '1': 0.0}",train,{'other annotations': {'aggressive language de...
783,"Better watch out Merkel, the German people don...",hate speech detection,6,"0,0,1,1,1,0","Ann1,Ann2,Ann3,Ann4,Ann5,Ann6",en,0,"{'0': 0.5, '1': 0.5}",train,{'other annotations': {'aggressive language de...


## Preprocess

In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(df[["text","label"]])# can be done directly from json as well

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [ ]:
def tokenize_function(examples):
    examples["label"] = int(examples["label"])#dataset stores label as string lol
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=False)

  0%|          | 0/784 [00:00<?, ?ex/s]

In [ ]:
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.1)

# Model and evaluation

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/pytorch_

In [ ]:
import numpy as np
import evaluate
metric = evaluate.load("f1")

In [ ]:
from sklearn.metrics import f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1_huggingface = metric.compute(predictions=predictions, references=labels)
    f1_sklearn = f1_score(labels, predictions, average = 'micro')  
    
    return {"f1_huggingface" : f1_huggingface, "f1_sklearn" : f1_sklearn}
#TODO do soft eval here

# Train

In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 705
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 267
  Number of trainable parameters = 124647170


Epoch,Training Loss,Validation Loss,F1 Huggingface,F1 Sklearn
1,No log,0.354564,{'f1': 0.0},0.886076
2,No log,0.379478,{'f1': 0.0},0.886076
3,No log,0.409790,{'f1': 0.0},0.886076


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 79
  Batch size = 8
Trainer is attempting to log a value of "{'f1': 0.0}" of type <class 'dict'> for key "eval/f1_huggingface" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 79
  Batch size = 8
Trainer is attempting to log a value of "{'f

TrainOutput(global_step=267, training_loss=0.3229807878701428, metrics={'train_runtime': 209.5722, 'train_samples_per_second': 10.092, 'train_steps_per_second': 1.274, 'total_flos': 556479882086400.0, 'train_loss': 0.3229807878701428, 'epoch': 3.0})

# Evaluation

not yet working

In [ ]:
def tokenize_function_eval(examples):
    examples["soft_label"] = int(examples["soft_label"])#dataset stores label as string lol
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [ ]:
import evaluate
import torch

dataset_eval = Dataset.from_pandas(df[["text","soft_label"]])
tokenized_datasets_eval = dataset_eval.map(tokenize_function_eval, batched=False)
dataloader_eval = torch.utils.data.DataLoader(dataset_eval, batch_size=32)
model.eval()
for batch in dataloader_eval:
    batch = {k: v.to("cuda") for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.clip(predictions, 0.01, 1 - 0.01)
    ce = -torch.sum(targets*torch.log(predictions+1e-9))/N

metric.compute()

In [ ]:
test